<a href="https://colab.research.google.com/github/raghavendra-gunnai/EVA4/blob/master/Session7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchsummary
!pip install torchvision

In [0]:
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(10)
batch_size = 128
EPOCHS=15
LAMBDA=0.1
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        ) # output_size = 30

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        ) # output_size = 28

        # CONVOLUTION BLOCK 2
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        ) # output_size = 26

        # CONVOLUTION BLOCK 3
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=230, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(230),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock1_1 = nn.Sequential(
            nn.Conv2d(in_channels=230, out_channels=32, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12
        
        # CONVOLUTION BLOCK 5
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=0.03)
        ) # output_size = 10

        # CONVOLUTION BLOCK 6
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 8

        # CONVOLUTION BLOCK 7
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 6

        #CONVOLUTION BLOCK 8
        self.convblock_depthwise = nn.Sequential(
            nn.Conv2d(in_channels = 256, out_channels=256, kernel_size=(3,3), groups=256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        )

        # TRANSITION BLOCK 2
        self.convblock1_2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=32, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 24
        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 8
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 4

        #DILATED CONVOLUTION BLOCK
        self.convblock_dilated = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), padding=2, dilation=(2,2), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        # CONVOLUTION BLOCK 9
        self.convblock9 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 4

        # TRANSITION BLOCK 3
        self.convblock1_3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=32, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 24
        self.pool3 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 10
        self.convblock10 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 4

        # CONVOLUTION BLOCK 11
        self.convblock11 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.Dropout(p=0.03) 
        ) # output_size = 4


        # OUTPUT BLOCK
        self.convblock1_4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            nn.Dropout(p=0.03)
        ) # output_size = 4

        self.avg = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
        ) # output_size = 1
        self.convblock_out = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 1

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.convblock1_1(x)
        x = self.pool1(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.convblock_depthwise(x)
        x = self.convblock1_2(x)
        x = self.pool2(x)
        x = self.convblock8(x)
        x = self.convblock_dilated(x)
        x = self.convblock9(x)
        x = self.convblock1_3(x)
        x = self.pool3(x)
        x = self.convblock10(x)
        x = self.convblock11(x)
        x = self.convblock1_4(x)
        x = self.avg(x)
        x = self.convblock_out(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             864
       BatchNorm2d-2           [-1, 32, 30, 30]              64
              ReLU-3           [-1, 32, 30, 30]               0
           Dropout-4           [-1, 32, 30, 30]               0
            Conv2d-5           [-1, 64, 28, 28]          18,432
       BatchNorm2d-6           [-1, 64, 28, 28]             128
              ReLU-7           [-1, 64, 28, 28]               0
           Dropout-8           [-1, 64, 28, 28]               0
            Conv2d-9          [-1, 128, 26, 26]          73,728
      BatchNorm2d-10          [-1, 128, 26, 26]             256
             ReLU-11          [-1, 128, 26, 26]               0
          Dropout-12          [-1, 128, 26, 26]               0
           Conv2d-13          [-1, 230, 24, 24]         264,960
      BatchNorm2d-14          [-1, 230,

In [0]:

from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()
    y_pred = model(data)

    # Calculate loss
    criteria = nn.CrossEntropyLoss()
    loss = criteria(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))




In [9]:
print(EPOCHS)

15


In [10]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,nesterov=False)
#scheduler = StepLR(optimizer, step_size=8, gamma=0.1)
for epoch in range(EPOCHS):
    #print('Epoch:', epoch+1,'LR:', scheduler.get_lr()[0])
    train(model, device, train_loader, optimizer, epoch)
    #scheduler.step()
    test(model, device, test_loader)

Loss=2.326693058013916 Batch_id=781 Accuracy=33.24: 100%|██████████| 782/782 [29:44<00:00,  1.76s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 1.5125, Accuracy: 4347/10000 (43.47%)



Loss=1.1362401247024536 Batch_id=781 Accuracy=50.13: 100%|██████████| 782/782 [29:33<00:00,  1.78s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 1.2692, Accuracy: 5453/10000 (54.53%)



Loss=1.1873944997787476 Batch_id=781 Accuracy=59.02: 100%|██████████| 782/782 [29:36<00:00,  1.77s/it]
  0%|          | 0/782 [00:00<?, ?it/s]

...


Test set: Average loss: 0.6582, Accuracy: 7886/10000 (78.86%)



Loss=0.8246609568595886 Batch_id=781 Accuracy=83.27: 100%|██████████| 782/782 [29:23<00:00,  1.78s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.6933, Accuracy: 7720/10000 (77.20%)



Loss=1.602508544921875 Batch_id=781 Accuracy=84.05: 100%|██████████| 782/782 [29:22<00:00,  1.77s/it]



Test set: Average loss: 0.6362, Accuracy: 7914/10000 (79.14%)



In [11]:
for epoch in range(5):
    #print('Epoch:', epoch+1,'LR:', scheduler.get_lr()[0])
    train(model, device, train_loader, optimizer, epoch)
    #scheduler.step()
    test(model, device, test_loader)

Loss=0.4816357493400574 Batch_id=781 Accuracy=84.56: 100%|██████████| 782/782 [29:22<00:00,  1.77s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.6481, Accuracy: 7867/10000 (78.67%)



Loss=0.6936810612678528 Batch_id=781 Accuracy=85.26: 100%|██████████| 782/782 [29:27<00:00,  1.78s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.6296, Accuracy: 7982/10000 (79.82%)



Loss=0.45810461044311523 Batch_id=781 Accuracy=86.29: 100%|██████████| 782/782 [29:26<00:00,  1.77s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.6248, Accuracy: 8008/10000 (80.08%)



Loss=0.19360080361366272 Batch_id=781 Accuracy=87.21: 100%|██████████| 782/782 [29:28<00:00,  1.79s/it]
  0%|          | 0/782 [00:00<?, ?it/s]


Test set: Average loss: 0.6262, Accuracy: 7994/10000 (79.94%)



Loss=1.0158114433288574 Batch_id=781 Accuracy=87.62: 100%|██████████| 782/782 [29:36<00:00,  1.78s/it]



Test set: Average loss: 0.6292, Accuracy: 8071/10000 (80.71%)

